In [45]:
import pandas as pd
import rdkit
from chembl_webresource_client.new_client import new_client
import numpy as np
from rdkit import Chem
from keras.models import Sequential
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle
from keras.layers import LeakyReLU
from rdkit.Chem import MACCSkeys
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from sklearn.model_selection import GridSearchCV
import time
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


In [2]:
df = pd.read_csv('test.csv')

In [3]:
act_normal = df[df.standard_value.notna()]
act_normal = act_normal[act_normal.canonical_smiles.notna()]
act_normal = act_normal.drop_duplicates(['canonical_smiles'])
act_normal

,Unnamed: 0,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,0,NaN,105501,[],CHEMBL666573,In vitro inhibition of human Dipeptidylpeptida...,B,NaN,NaN,BAO_0000190,...,Homo sapiens,Dipeptidyl peptidase IV,9606,NaN,NaN,IC50,uM,UO_0000065,NaN,217.0
1,1,NaN,106644,[],CHEMBL666573,In vitro inhibition of human Dipeptidylpeptida...,B,NaN,NaN,BAO_0000190,...,Homo sapiens,Dipeptidyl peptidase IV,9606,NaN,NaN,IC50,uM,UO_0000065,NaN,41.0
2,2,NaN,106647,[],CHEMBL666573,In vitro inhibition of human Dipeptidylpeptida...,B,NaN,NaN,BAO_0000190,...,Homo sapiens,Dipeptidyl peptidase IV,9606,NaN,NaN,IC50,uM,UO_0000065,NaN,15.0
3,3,NaN,106650,[],CHEMBL666573,In vitro inhibition of human Dipeptidylpeptida...,B,NaN,NaN,BAO_0000190,...,Homo sapiens,Dipeptidyl peptidase IV,9606,NaN,NaN,IC50,uM,UO_0000065,NaN,500.0
4,4,NaN,108924,[],CHEMBL666573,In vitro inhibition of human Dipeptidylpeptida...,B,NaN,NaN,BAO_0000190,...,Homo sapiens,Dipeptidyl peptidase IV,9606,NaN,NaN,IC50,uM,UO_0000065,NaN,188.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5045,5045,NaN,23308670,[],CHEMBL4843583,Inhibition of SF-tagged DPP4 (unknown origin) ...,B,NaN,NaN,BAO_0000190,...,Homo sapiens,Dipeptidyl peptidase IV,9606,NaN,NaN,IC50,nM,UO_0000065,NaN,4000.0
5046,5046,NaN,23308671,[],CHEMBL4843583,Inhibition of SF-tagged DPP4 (unknown origin) ...,B,NaN,NaN,BAO_0000190,...,Homo sapiens,Dipeptidyl peptidase IV,9606,NaN,NaN,IC50,nM,UO_0000065,NaN,4000.0
5047,5047,NaN,23308672,[],CHEMBL4843583,Inhibition of SF-tagged DPP4 (unknown origin) ...,B,NaN,NaN,BAO_0000190,...,Homo sapiens,Dipeptidyl peptidase IV,9606,NaN,NaN,IC50,nM,UO_0000065,NaN,4000.0
5048,5048,NaN,23308673,[],CHEMBL4843583,Inhibition of SF-tagged DPP4 (unknown origin) ...,B,NaN,NaN,BAO_0000190,...,Homo sapiens,Dipeptidyl peptidase IV,9606,NaN,NaN,IC50,nM,UO_0000065,NaN,4000.0


In [4]:
selection = ['molecule_chembl_id', 'canonical_smiles', 'standard_value']
act_normal = act_normal.reset_index()
act_normal = act_normal[selection]
act_normal


,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL93558,N[C@@H](CC1CCCCC1)C(=O)N1CCCCC1,217000.0
1,CHEMBL443622,C[C@H](N)C(=O)N1CCCC1,41000.0
2,CHEMBL403882,O=C([C@@H]1CCCN1)N1CCCC1,15000.0
3,CHEMBL328655,S=C(C1CCCN1)N1CCCC1,500000.0
4,CHEMBL328795,NC(=O)CC(N)C(=O)N1CCCC1,188000.0
...,...,...,...
4282,CHEMBL4868869,O=C(NCC(=O)N1CCOCC1)C(=O)[C@@H]1CCCN1C(=O)CNC(...,4000.0
4283,CHEMBL4871602,CC(C)(C)NC(=O)CNC(=O)C(=O)[C@@H]1CCCN1C(=O)CNC...,4000.0
4284,CHEMBL4854865,COC(=O)[C@H](Cc1ccccc1)NC(=O)CNC(=O)C(=O)[C@@H...,4000.0
4285,CHEMBL4864489,COC(=O)[C@H](CCC(=O)O)NC(=O)CNC(=O)C(=O)[C@@H]...,4000.0


In [5]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', 1)
        
    return x


In [6]:
act_normal = norm_value(act_normal)

/tmp/ipykernel_306790/1998879194.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x = input.drop('standard_value', 1)


In [7]:
def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', 1)
        
    return x


In [8]:
act_final = pIC50(act_normal)
act_final


/tmp/ipykernel_306790/4026072073.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x = input.drop('standard_value_norm', 1)


,molecule_chembl_id,canonical_smiles,pIC50
0,CHEMBL93558,N[C@@H](CC1CCCCC1)C(=O)N1CCCCC1,3.663540
1,CHEMBL443622,C[C@H](N)C(=O)N1CCCC1,4.387216
2,CHEMBL403882,O=C([C@@H]1CCCN1)N1CCCC1,4.823909
3,CHEMBL328655,S=C(C1CCCN1)N1CCCC1,3.301030
4,CHEMBL328795,NC(=O)CC(N)C(=O)N1CCCC1,3.725842
...,...,...,...
4282,CHEMBL4868869,O=C(NCC(=O)N1CCOCC1)C(=O)[C@@H]1CCCN1C(=O)CNC(...,5.397940
4283,CHEMBL4871602,CC(C)(C)NC(=O)CNC(=O)C(=O)[C@@H]1CCCN1C(=O)CNC...,5.397940
4284,CHEMBL4854865,COC(=O)[C@H](Cc1ccccc1)NC(=O)CNC(=O)C(=O)[C@@H...,5.397940
4285,CHEMBL4864489,COC(=O)[C@H](CCC(=O)O)NC(=O)CNC(=O)C(=O)[C@@H]...,5.397940


In [9]:
bioactivity_threshold = []

for i in act_final.pIC50:
  if float(i) >= 7:
     bioactivity_threshold.append('ACTIVE')
  elif float(i) <= 6:
     bioactivity_threshold.append('INACTIVE')
  else:
    bioactivity_threshold.append('INTERMEDIATE')
bioactivity_class = pd.Series(bioactivity_threshold, name = 'bioactivity_class')
act5 = pd.concat([act_final, bioactivity_class], axis=1)
act5

,molecule_chembl_id,canonical_smiles,pIC50,bioactivity_class
0,CHEMBL93558,N[C@@H](CC1CCCCC1)C(=O)N1CCCCC1,3.663540,INACTIVE
1,CHEMBL443622,C[C@H](N)C(=O)N1CCCC1,4.387216,INACTIVE
2,CHEMBL403882,O=C([C@@H]1CCCN1)N1CCCC1,4.823909,INACTIVE
3,CHEMBL328655,S=C(C1CCCN1)N1CCCC1,3.301030,INACTIVE
4,CHEMBL328795,NC(=O)CC(N)C(=O)N1CCCC1,3.725842,INACTIVE
...,...,...,...,...
4282,CHEMBL4868869,O=C(NCC(=O)N1CCOCC1)C(=O)[C@@H]1CCCN1C(=O)CNC(...,5.397940,INACTIVE
4283,CHEMBL4871602,CC(C)(C)NC(=O)CNC(=O)C(=O)[C@@H]1CCCN1C(=O)CNC...,5.397940,INACTIVE
4284,CHEMBL4854865,COC(=O)[C@H](Cc1ccccc1)NC(=O)CNC(=O)C(=O)[C@@H...,5.397940,INACTIVE
4285,CHEMBL4864489,COC(=O)[C@H](CCC(=O)O)NC(=O)CNC(=O)C(=O)[C@@H]...,5.397940,INACTIVE


In [10]:
act5 = act5[act5['bioactivity_class'] != 'INTERMEDIATE']


In [11]:
act5 = act5.reset_index()

In [12]:
chembl_df = act5[["canonical_smiles", "pIC50"]]
chembl_df.head()


,canonical_smiles,pIC50
0,N[C@@H](CC1CCCCC1)C(=O)N1CCCCC1,3.663540
1,C[C@H](N)C(=O)N1CCCC1,4.387216
2,O=C([C@@H]1CCCN1)N1CCCC1,4.823909
3,S=C(C1CCCN1)N1CCCC1,3.301030
4,NC(=O)CC(N)C(=O)N1CCCC1,3.725842


In [16]:
def smiles_to_fp(smiles, method="maccs", n_bits=2048):
    # Convert smiles to RDKit mol object
    mol = Chem.MolFromSmiles(smiles)

    if method == "maccs":
        return np.array(MACCSkeys.GenMACCSKeys(mol))
    if method == "morgan2":
        return np.array(GetMorganFingerprintAsBitVect(mol, 2, nBits=n_bits))
    if method == "morgan3":
        return np.array(GetMorganFingerprintAsBitVect(mol, 3, nBits=n_bits))
    else:
        print(f"Warning: Wrong method specified: {method}." " Default will be used instead.")
        return np.array(MACCSkeys.GenMACCSKeys(mol))


In [17]:
chembl_df["fingerprints_df"] = chembl_df["canonical_smiles"].apply(smiles_to_fp)

# Look at head
print("Shape of dataframe:", chembl_df.shape)
chembl_df.head(3)


Shape of dataframe: (3400, 3)


/tmp/ipykernel_306790/3840200718.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chembl_df["fingerprints_df"] = chembl_df["canonical_smiles"].apply(smiles_to_fp)


,canonical_smiles,pIC50,fingerprints_df
0,N[C@@H](CC1CCCCC1)C(=O)N1CCCCC1,3.663540,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,C[C@H](N)C(=O)N1CCCC1,4.387216,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,O=C([C@@H]1CCCN1)N1CCCC1,4.823909,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [23]:
bioactivity_threshold = []

for i in chembl_df.pIC50:
  if float(i) >= 7:
     bioactivity_threshold.append('ACTIVE')
  elif float(i) <= 6:
     bioactivity_threshold.append('INACTIVE')
  else:
    bioactivity_threshold.append('INTERMEDIATE')
bioactivity_class = pd.Series(bioactivity_threshold, name = 'bioactivity_class')
act5 = pd.concat([chembl_df, bioactivity_class], axis=1)
act5

,canonical_smiles,pIC50,fingerprints_df,bioactivity_class
0,N[C@@H](CC1CCCCC1)C(=O)N1CCCCC1,3.663540,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",INACTIVE
1,C[C@H](N)C(=O)N1CCCC1,4.387216,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",INACTIVE
2,O=C([C@@H]1CCCN1)N1CCCC1,4.823909,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",INACTIVE
3,S=C(C1CCCN1)N1CCCC1,3.301030,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",INACTIVE
4,NC(=O)CC(N)C(=O)N1CCCC1,3.725842,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",INACTIVE
...,...,...,...,...
3395,O=C(NCC(=O)N1CCOCC1)C(=O)[C@@H]1CCCN1C(=O)CNC(...,5.397940,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",INACTIVE
3396,CC(C)(C)NC(=O)CNC(=O)C(=O)[C@@H]1CCCN1C(=O)CNC...,5.397940,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",INACTIVE
3397,COC(=O)[C@H](Cc1ccccc1)NC(=O)CNC(=O)C(=O)[C@@H...,5.397940,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",INACTIVE
3398,COC(=O)[C@H](CCC(=O)O)NC(=O)CNC(=O)C(=O)[C@@H]...,5.397940,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",INACTIVE


In [65]:
act5[300:340]

,canonical_smiles,pIC50,fingerprints_df,bioactivity_class
300,N[C@@H](CCNC1CCCCC1)C(=O)N1CCCCC1,3.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
301,N[C@@H](CCN(Cc1ccccc1)Cc1ccccc1)C(=O)N1CCCCC1,3.575118,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
302,N[C@@H](CCNC1CCNCC1)C(=O)N1CCCCC1,4.193820,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
303,N[C@@H](CCNCc1ccc(Cl)cc1)C(=O)N1CCC(O)CC1,3.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
304,N#C[C@@H]1CCCCN1C(=O)[C@@H](N)CCN,3.821023,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
305,N[C@@H](CCNc1ncccc1[N+](=O)[O-])C(=O)N1CCCCC1,4.251812,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
306,N#Cc1ccc(NCC[C@H](N)C(=O)N2CCCCC2)nc1,3.688246,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
307,NCC[C@H](N=C(N)N)C(=O)N1CCCCC1,3.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
308,N[C@@H](CCNCc1ccc(Cl)cc1)C(=O)N1CCCC(O)C1,3.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
309,COc1ccccc1CNCC[C@H](N)C(=O)N1CCCCC1,3.301030,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


In [28]:
act5 = act5[act5['bioactivity_class'] != 'INTERMEDIATE']


In [30]:
act5['bioactivity_class'] = act5['bioactivity_class'].map(lambda x: 0 if x == 'ACTIVE' else 1)

In [31]:
x_train, x_test, y_train, y_test = train_test_split(
    act5["fingerprints_df"], act5[["bioactivity_class"]], test_size=0.2, random_state=42
)

# Print the shape of training and testing data
print("Shape of training data:", x_train.shape)
print("Shape of test data:", x_test.shape)


Shape of training data: (2720,)
Shape of test data: (680,)


In [33]:
from sklearn.ensemble import RandomForestClassifier

x_train, x_test, y_train, y_test = train_test_split(
    act5["fingerprints_df"], act5[["bioactivity_class"]],test_size=0.33,random_state=42
)

In [34]:
param_grid = { 
    'n_estimators': [100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
rfc=RandomForestClassifier(random_state=42)


In [37]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)

In [38]:
random_forrest = CV_rfc.fit(np.array(list((x_train))),y_train.values)

/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.1

/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: Dat

/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.1

/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.1

/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: Dat

...

/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: Dat

/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.1

/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.1

/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: Dat

/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/longvudang/.local/lib/python3.1

In [47]:
start_time=time.time()
training_time_random_forrest=time.time()-start_time

random_forrest_train_rscore=random_forrest.score(np.array(list((x_train))),y_train.values)
start_time2=time.time()
random_forrest_train_pred=random_forrest.predict(np.array(list((x_train))))
testing_time=time.time()-start_time2
random_forrest_train_rmse=np.sqrt(mean_squared_error(y_train.values,random_forrest_train_pred))
print("R squared for Training data: ", random_forrest_train_rscore)
print("Root Mean Squared Error for Training data: ", random_forrest_train_rmse)


R squared for Training data:  0.8977172958735733
Root Mean Squared Error for Training data:  0.31981667268362773


In [48]:
random_forrest_test_rscore=random_forrest.score(np.array(list((x_test))),y_test.values)
start_time2=time.time()
random_forrest_test_pred=random_forrest.predict(np.array(list((x_test))))
testing_time_random_forrest=time.time()-start_time2
random_forrest_test_rmse=np.sqrt(mean_squared_error(y_test.values,random_forrest_test_pred))
random_forrest_test_mae=mean_absolute_error(y_test.values,random_forrest_test_pred)


In [51]:
def RF():
  return {'Regressor Model':'Random Forrest',
          'Predictions':random_forrest_test_pred,
          'R Squared': random_forrest_test_rscore,
          'Mean Absolute Error':random_forrest_test_mae,
          'Root Mean Squared Error':random_forrest_test_rmse,
          'Training Time': training_time_random_forrest,
          'Prediction Time':testing_time}

## SVM

In [52]:
x_train, x_test, y_train, y_test = train_test_split(
    act5["fingerprints_df"], act5[["bioactivity_class"]], test_size=0.2, random_state=42
)

# Print the shape of training and testing data
print("Shape of training data:", x_train.shape)
print("Shape of test data:", x_test.shape)


Shape of training data: (2720,)
Shape of test data: (680,)


In [53]:
param_grid = {
 'C': (np.arange(0.1,1,0.1)) , 'kernel': ['linear'],
 'C': (np.arange(0.1,1,0.1)) , 'gamma': [0.01,0.02,0.03,0.04,0.05], 'kernel': ['rbf'],
 'degree': [2,3,4] ,'gamma':[0.01,0.02,0.03,0.04,0.05], 'C':(np.arange(0.1,1,0.1)) , 'kernel':['poly']
                   }
# param_grid = {'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}
grid_search = GridSearchCV(SVC(C=0.1), param_grid, cv=10)
model_svm= grid_search.fit(np.array(list((x_train))),y_train.values)


/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

...

/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/longvudang/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

In [54]:
print("Best kernel: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Best kernel:  {'C': 0.2, 'degree': 4, 'gamma': 0.05, 'kernel': 'poly'}
Best score:  0.9018382352941178


In [55]:
start_time=time.time()
training_time_svm=time.time()-start_time


In [56]:
rbf_svc_train_rscore=model_svm.score(np.array(list((x_train))),y_train.values)
rbf_svc_train_pred=model_svm.predict(np.array(list((x_train))))
rbf_svc_train_rmse=np.sqrt(mean_squared_error(y_train.values,rbf_svc_train_pred))


In [57]:
rbf_svc_train_rscore=model_svm.score(np.array(list((x_train))),y_train.values)
print("R squared for Training data: ",rbf_svc_train_rscore)
print("Root Mean Squared Error for Training data: ", rbf_svc_train_rmse)


R squared for Training data:  0.9680147058823529
Root Mean Squared Error for Training data:  0.17884432928568647


In [58]:
rbf_svc_test_rscore=model_svm.score(np.array(list((x_test))),y_test.values)
start_time2=time.time()
rbf_svc_test_pred=model_svm.predict(np.array(list((x_test))))
testing_time_svm=time.time()-start_time2
rbf_svc_test_rmse=np.sqrt(mean_squared_error(y_test.values,rbf_svc_test_pred))
rbf_svc_test_mae=mean_absolute_error(y_test.values,rbf_svc_test_pred)

In [59]:
print("R squared for Testing data: ", rbf_svc_test_rscore)
print("Root Mean Squared Error for Testing data: ", rbf_svc_test_rmse)
print("Mean Absolute Error for Testing data: ", rbf_svc_test_mae)


R squared for Testing data:  0.9088235294117647
Root Mean Squared Error for Testing data:  0.30195441806377876
Mean Absolute Error for Testing data:  0.09117647058823529


In [60]:
def SVM():
  return {'Regressor Model':'SVM Regressor',
          'Predictions':rbf_svc_test_pred,
          'R Squared': rbf_svc_test_rscore,
          'Mean Absolute Error':rbf_svc_test_mae,
          'Root Mean Squared Error':rbf_svc_test_rmse,
          'Training Time': training_time_svm,
          'Prediction Time':testing_time_svm}

In [61]:
models={}
models[1]=RF()
models[2]=SVM()


In [62]:
model_table=pd.DataFrame.from_dict(models,orient='index')
model_table


,Regressor Model,Predictions,R Squared,Mean Absolute Error,Root Mean Squared Error,Training Time,Prediction Time
1,Random Forrest,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",0.860963,0.139037,0.372877,0.000035,0.023487
2,SVM Regressor,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",0.908824,0.091176,0.301954,0.000027,0.037901


In [63]:
a= model_svm.predict(np.array(list((x_test))))

In [64]:
filename = 'svm.pkl'
pickle.dump(model_svm, open(filename, 'wb'))
